In [1]:
import tensorflow as tf
import pickle as pickle
import numpy as np
import math
import os

In [2]:
data_path = ""
img_size = 32
num_channels = 3
img_size_flat = img_size * img_size * num_channels
num_classes = 10
_num_files_train = 5
_images_per_file = 10000
_num_images_train = _num_files_train * _images_per_file

In [3]:
def _unpickle(filename):  
    """
    주어진 파일을 unpickle하고 data를 return 함
    """
    file_path = os.path.join(data_path, "cifar-10-batches-py/", filename)

    print("Loading data: " + file_path)

    with open(file_path, mode='rb') as file:
        # In Python 3.X it is important to set the encoding,
        # otherwise an exception is raised here.
        data = pickle.load(file, encoding='bytes')

    return data

In [4]:
def one_hot_encoded(class_numbers, num_classes=None):
    """
    class_label들을 one_hot_encode로 생성해줌
    예를 들어, class_number=2 그리고 num_classes=4 면
    one-hot eoncoded label 은 [0, 0, 1, 0]
    
    :param class_numbers:
        class_numbers의 integer 배열
    :param_classes:
        classes의 수
    :return
        2차원 shape [len(cls), num_classes]
    """
    if num_classes is None:
        num_classes = np.max(class_numbers) - 1

    return np.eye(num_classes, dtype=float)[class_numbers]


def _convert_images(raw):
    """
    CIFAR-10 format으로 부터 image를 convert 하고
    4차원 array로 return [image_number, height, width, channel]
    pixels 0.0과 1.0사이에 floats
    """
    raw_float = np.array(raw, dtype=float) / 255.0
    # Reshape the array to 4-dimensions.
    images = raw_float.reshape([-1, num_channels, img_size, img_size])
    # Reorder the indices of the array.
    images = images.transpose([0, 2, 3, 1])

    return images

def _load_data(filename): 
    """
    pickled data file을 load함
    그리고 converted image와 각 image의 class-number를 return함
    """
    data = _unpickle(filename)
    raw_images = data[b'data']
    cls = np.array(data[b'labels'])
    images = _convert_images(raw_images)
    return images, cls

def load_class_names():
    """
    cifar10 data set에 class들에 names를 load함
    이름 list를 return 함
    예를 들면 name[3] 은 class-number 3
    """
    raw = _unpickle(filename="batches.meta")[b'label_names']
    names = [x.decode('utf-8') for x in raw]
    return names


def load_training_data(): 
    """
    cifar10의 모든 training data를 load함
    data set은 5개의 파일로 나눠져 있고 여기서 merge함
    image, class -number, one-hot encode된 class-label을 return 함
    """
    images = np.zeros(shape=[_num_images_train, img_size, img_size, num_channels], dtype=float)
    cls = np.zeros(shape=[_num_images_train], dtype=int)

    begin = 0
    for i in range(_num_files_train):
        images_batch, cls_batch = _load_data(filename="data_batch_" + str(i + 1))
        
        num_images = len(images_batch)
        end = begin + num_images
        images[begin:end, :] = images_batch
        cls[begin:end] = cls_batch
        begin = end

    return images, cls, one_hot_encoded(class_numbers=cls, num_classes=num_classes)


def load_test_data():
    """
    모든 test-data set을 load함
    image , class-numbers and one-hot encode된 class-labels를 return 한다.
    """
    images, cls = _load_data(filename="test_batch")
    return images, cls, one_hot_encoded(class_numbers=cls, num_classes=num_classes)



In [5]:
def xavier_init(n_inputs, n_outputs, uniform=True):  
    """
    param n_inputs:
        input nodes의 수
    param n_outputs:
        output nodes의 수
    param uniform:
        true라면 uniform distribution을 사용 , 아니면 normal을 사용
    """
    
    if uniform:
        init_range = math.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = math.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)
    
    
def variable_xavier(name,shape):
    var = tf.get_variable(name = name, shape = shape , initializer=tf.contrib.layers.xavier_initializer())
    return var
    
def restoreSaver(session,check_dir):
    """
    (*) 실행하기 전 last_epoch = tf.Variable(0, name='last_epoch') 선언
    (*) session.run(global_variables_initializer()) 하기 전에 선언시켜줘야함.
    (*) 복귀된 epoch는 session.run(last_epoch)의 리턴값으로 얻어짐.
        Restore Saver
    :param session:
        현재 실행시킨 session
    :param check_dir:
        저장된 디렉토리 경로
    :return:
        세이버 리턴
    """
    saver = tf.train.Saver()
    check_point = tf.train.get_checkpoint_state(check_dir)

    if check_point and check_point.model_checkpoint_path:
        try:
            saver.restore(session, check_point.model_checkpoint_path)
            print('successfully loaded : ', check_point.model_checkpoint_path)
        except:
            print('fail to load')
    else:
        print('could not find load data')

    return saver

def saveSaver(session, last_epoch ,global_step ,saver, check_dir, model_name = 'model'):
    """
        체크포인트 세이브
    :param session:
        현재 세션
    :param last_epoch:
        마지막 epoch
        last_epoch - tf.Variable
    :param global_step:
        현재 global_step
    :param saver:
        저장시킬 saver
    :param check_dir:
        저장시킬 directory - restoreSaver 의 check_dir 과 같음.
    :param model_name:
        저장시킬 모델이름.
        default = 'model'
    :return:
        세이버 리턴
    """
    import os
    if not os.path.exists(check_dir):
        os.makedirs(check_dir)
    try:
        session.run(last_epoch.assign(global_step+1))
        saver.save(sess=session,save_path = check_dir+'/'+model_name, global_step = global_step)
        print('step',global_step,'successfully save dir :', check_dir+'/'+model_name+'-'+str(global_step))
    except:
        print('fail to save')

    return saver

In [7]:
class batch_norm(object): 
    """
    batch_normalization class
    epsilon은 1e-5
    momentum 0.9로 초기화 하고 씀
    __call__함수를 정의해서 사용할 때 변수에 call 해서 사용할 수 있음
    """

    def __init__(self, epsilon=1e-5, momentum=0.9, name="batch_norm")
        self.epsilon = epsilon
        self.momentum = momentum
        self.name = name

    def __call__(self, x, train=True):
        
        return tf.contrib.layers.batch_norm(x,
                                        decay=self.momentum,
                                        updates_collections=None,
                                        epsilon=self.epsilon,
                                        scale=True,
                                        is_training=train,
                                        scope=self.name)


def conv2d(t_input,ksize,strides,padding,layerName,initializer=variable_xavier):
    """
    convolution function
    param t_input:
        convolution하기 위해 들어오는 input
    param ksize:
        filter size [x, x, x ,x]
    param strides:
        strides [1,x,x,1]
    param layerName:
        레이어 이름
    param initializer:
        default로 xavier를 사용해서 initialization
    """
    W = initializer(name=layerName+"/Weight", shape=ksize)
    output = tf.nn.conv2d(input = t_input, filter = W, strides = strides, padding = padding)
    return output


def maxPool(t_input,ksize,strides,padding,layerName):
    """
     param t_input:
        max_pooling 하기 위해 들어오는 input
    param ksize:
        filter size [x, x, x ,x]
    param strides:
        strides [1,x,x,1]
    param layerName:
        레이어 이름
    """
    output = tf.nn.max_pool(t_input, ksize = ksize, strides=strides, padding = padding)
    return output


def flatten(t_input, flatDim, layerName='flattenLayer'):
    """
    t_input을 [-1,flatDim] 모양으로 reshape 해줌
    """
    flatten = tf.reshape(t_input, [-1, flatDim])
    return flatten

def fullyConnected(t_input, shape, layerName, initializer = variable_xavier): 
    """
    W와 B를 xavier를 이용해서 initializaion 해줌
    fully connected로 만들어줌
    """
    W = initializer(name=layerName+'/weight',shape = shape)
    B = initializer(name=layerName+'/bias',shape = shape[-1])
    preActivate = tf.matmul(t_input, W) + B
    return preActivate

def crossEntropy(labels, logits, name = 'lossFunction'):
    """
    cross_entropy에 reduce_mean을 해서 return 해줌
    """
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = labels, logits = logits))
    return cost

def lrelu(x,leak=0.2):
    """
    leakyRelu를 return 해줌
    """
    return tf.maximum(x,leak*x)

def trainOptimizer(cost, learning_rate = 1e-3, optimizer = tf.train.AdamOptimizer, name = 'train'):
    """
    train할 optimizer를 정해줌
    여기서는 learning late를 1e-3으로 하고
    adamOptimizer를 사용함
    다른 optimizer로 바꿀 수 있음
    """
    optimize = optimizer(learning_rate = learning_rate).minimize(cost)
    return optimize


In [ ]:
class VGG19:  #vgg19 layers
    def __init__(self, input, t):
        if input is None: return
        self.out, self.phi = self.build_model(input)
        self.loss = self.inference_loss(self.out, t)

    def build_model(self, t_input, reuse=False):
            phi = []
            with tf.variable_scope('conv1a'):      #[None, 32, 32, 3]
                conv1a = conv2d(
                                layerName='vgg19_conv1a',
                                t_input=t_input,        #처음에 들어가는 input
                                ksize= [3, 3, 3, 64],   #filter size는 3x3,  InChennel =3, outChennel =64
                                strides=[1,1,1,1],      #stride =1
                                padding='SAME')
                BN = batch_norm()      #Batch_norm을 BN으로 사용할 것임
                conv1a = BN(conv1a)    #con1a에 bach_norm을 적용
                conv1a = lrelu(conv1a)   #bach_norm에서 나온 값을 leakyRelu activation function을 사용함
                print("conv1a :", conv1a)  #[None, 32, 32, 64]


            with tf.variable_scope('conv1b'):     
                conv1b = conv2d(layerName='vgg19_conv1b',
                                t_input=conv1a,
                                ksize=[3,3,64,64],
                                strides=[1,1,1,1],
                                padding='SAME')
                BN = batch_norm()
                conv1b =BN(conv1b)
                conv1b =lrelu(conv1b)
                print("conv1b :", conv1b)     #[None, 32, 32, 64]
            phi.append(conv1b)

            pool_first = maxPool(layerName='pool_first', #첫번째 max_pooling
                                 t_input=conv1b,            
                                 ksize=[1,2,2,1],        #ksize= 2X2
                                 strides=[1,2,2,1],      #stride = 2
                                 padding='SAME')        
            
            print("max_pool_first :",pool_first)       #[None, 16, 16, 128]
            # pool 1

            with tf.variable_scope('conv2a'):
                conv2a = conv2d(layerName='vgg19_conv2a',
                                t_input=pool_first,
                                ksize=[3, 3, 64, 128],
                                strides=[1,1,1,1],
                                padding='SAME')

                BN = batch_norm()
                conv2a = BN(conv2a)
                conv2a = lrelu(conv2a)
                print("conv2a :", conv2a)            #[None, 16, 16, 128]


            with tf.variable_scope('conv2b'):
                conv2b = conv2d(layerName='vgg19_conv2b',
                                t_input=conv2a,
                                ksize=[3, 3, 128, 128],
                                strides=[1,1,1,1],
                                padding='SAME')
                BN = batch_norm()
                conv2b = BN(conv2b)
                conv2b = lrelu(conv2b)
                print("conv2b :", conv2b)         #[None, 16, 16, 128]
            phi.append(conv2b)

            pool_second=maxPool(layerName='pool_first',
                                t_input=conv2b,
                                ksize=[1, 2, 2, 1],
                                strides=[1, 2, 2, 1],
                                padding='SAME')
            
            print("max_pool_second :",pool_second)    #[None, 8, 8, 128]
            # pool 2

            with tf.variable_scope('conv3a'):
                conv3a = conv2d(layerName='vgg19_conv3a',
                                t_input=pool_second,
                                ksize=[3, 3, 128, 256],
                                strides=[1,1,1,1],
                                padding='SAME')

                BN = batch_norm()
                conv3a = BN(conv3a)
                conv3a = lrelu(conv3a)
                print("conv3a :", conv3a)       #[None, 8, 8, 256]


            with tf.variable_scope('conv3b'):
                conv3b =conv2d(layerName='vgg19_conv3b',
                                       t_input=conv3a,
                                       ksize=[3, 3, 256, 256],
                                       strides=[1,1,1,1],
                                       padding='SAME')

                BN = batch_norm()
                conv3b = BN(conv3b)
                conv3b = lrelu(conv3b)
                print("conv3b :", conv3b)

            with tf.variable_scope('conv3c'):
                conv3c = conv2d(layerName='vgg19_conv3c',
                                       t_input=conv3b,
                                       ksize=[3, 3, 256, 256],
                                       strides=[1,1,1,1],
                                       padding='SAME')
                BN = batch_norm()
                conv3c = BN(conv3c)
                conv3c = lrelu(conv3c)
                print("conv3c :", conv3c)      #[None, 8, 8, 256]

            with tf.variable_scope('conv3d'):
                conv3d = conv2d(layerName='vgg19_conv3d',
                                       t_input=conv3c,
                                       ksize=[3, 3, 256, 256],
                                       strides=[1,1,1,1],
                                       padding='SAME')
                BN = batch_norm()
                conv3d = BN(conv3d)
                conv3d = lrelu(conv3d)
                print("conv3d :", conv3d)       #[None, 8, 8, 256]
            phi.append(conv3d)

            pool_third =maxPool(layerName='pool_first',
                               t_input=conv3d,
                               ksize=[1, 2, 2, 1],
                                strides=[1, 2, 2, 1],
                                padding='SAME')
            
            print("max_pool_third :",pool_third)  #[None, 4, 4, 256]

            with tf.variable_scope('conv4a'):
                conv4a = conv2d(layerName='vgg19_conv4a',
                                       t_input=pool_third,
                                       ksize=[3, 3, 256, 512],
                                       strides=[1,1,1,1],
                                       padding='SAME')
                BN = batch_norm()
                conv4a = BN(conv4a)
                conv4a = lrelu(conv4a)
                print("conv4a :", conv4a)      #[None, 4, 4, 512]

            with tf.variable_scope('conv4b'):
                conv4b = conv2d(layerName='vgg19_conv4b',
                                       t_input=conv4a,
                                       ksize=[3, 3, 512, 512],
                                       strides=[1, 1, 1, 1],
                                       padding='SAME')
                BN = batch_norm()
                conv4b = BN(conv4b)
                conv4b = lrelu(conv4b)
                print("conv4b :", conv4b)    #[None, 4, 4, 512]


            with tf.variable_scope('conv4c'):
                conv4c = conv2d(layerName='vgg19_conv4c',
                                       t_input=conv4b,
                                       ksize=[3, 3, 512, 512],
                                       strides=[1, 1, 1, 1],
                                       padding='SAME')
                BN = batch_norm()
                conv4c = BN(conv4c)
                conv4c = lrelu(conv4c)
                print("conv4c :", conv4c)    #[None, 4, 4, 512]


            with tf.variable_scope('conv4d'):
                conv4d = conv2d(layerName='vgg19_conv4d',
                                       t_input=conv4c,
                                       ksize=[3, 3, 512, 512],
                                       strides=[1, 1, 1, 1],
                                       padding='SAME')
                BN = batch_norm()
                conv4d = BN(conv4d)
                conv4d = lrelu(conv4d)
                print("conv4d :", conv4d)     #[None, 4, 4, 512]

            phi.append(conv4d)

            pool_fourth = maxPool(layerName='pool_first',
                                  t_input=conv4d,
                                  ksize=[1, 2, 2, 1],
                                  strides=[1, 2, 2, 1],
                                  padding='SAME')
            print("max_pool_fourth :",pool_fourth)     #[None, 2, 2, 512]

            with tf.variable_scope('conv5a'):
                conv5a = conv2d(layerName='vgg19_conv5a',
                                       t_input=pool_fourth,
                                       ksize=[3, 3, 512, 512],
                                       strides=[1, 1, 1, 1],
                                       padding='SAME')
                BN = batch_norm()
                conv5a = BN(conv5a)
                conv5a = lrelu(conv5a)
                print("conv5a :", conv5a)   #[None, 2, 2, 512]

            with tf.variable_scope('conv5b'):
                conv5b = conv2d(layerName='vgg19_conv5b',
                                t_input=conv5a,
                                 ksize=[3, 3, 512, 512],
                                strides=[1, 1, 1, 1],
                                padding='SAME')
                BN = batch_norm()
                conv5b = BN(conv5b)
                conv5b = lrelu(conv5b)
                print("conv5b :", conv5b)   #[None, 2, 2, 512]

            with tf.variable_scope('conv5c'):
                conv5c = conv2d(layerName='vgg19_conv5c',
                                t_input=conv5b,
                                ksize=[3, 3, 512, 512],
                                strides=[1, 1, 1, 1],
                                padding='SAME')
                BN = batch_norm()
                conv5c = BN(conv5c)
                conv5c = lrelu(conv5c)
                print("conv5c :", conv5c)   #[None, 2, 2, 512]

            with tf.variable_scope('conv5d'):
                conv5d = conv2d(layerName='vgg19_conv5d',
                                t_input=conv5c,
                                ksize=[3, 3, 512, 512],
                                strides=[1, 1, 1, 1],
                                padding='SAME')
                BN = batch_norm()
                conv5d = BN(conv5d)
                conv5d = lrelu(conv5d)
                print("conv5d :", conv5d)  #[None, 2, 2, 512]

            phi.append(conv5d)

            pool_fifth = maxPool(layerName='pool_first',
                                 t_input=conv5d,
                                 ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1],
                                 padding='SAME')
            print("max_pool_fifth :",pool_fifth)   #[None, 1, 1, 512]

            flat = flatten(t_input=pool_fifth, flatDim=512)      #fully_connected를 하기전에 flatten을 해줌
            print("flatten :", flat)                    #[None,512]

            with tf.variable_scope('fc1'):
                fc1 = fullyConnected(flat, [512, 4096], layerName='fc1')     #[None, 512]를 -> [None,4096]
                fc1 = lrelu(fc1)
                print("fc1 :",fc1)
                
            with tf.variable_scope('fc2'):
                fc2 = fullyConnected(fc1, [4096, 4096], layerName='fc2')     #[None ,4096] -> [None,4096]
                fc2 = lrelu(fc2)
                print("fc2 :",fc2)
                
            with tf.variable_scope('softmax'):
                fc3 = fullyConnected(fc2, [4096, 10], layerName='fc3')      #[None ,4096] -> [None,10]
                print("fc3 :",fc3)
                
            return fc3, phi


    def inference_loss(self, out, t):     #loss function
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=t,logits=out)
        return tf.reduce_mean(cross_entropy)


In [ ]:
X= tf.placeholder(tf.float32,shape=[None,32,32,3])   #input
Y_label = tf.placeholder(tf.float32,shape=[None,10])  #Y_label

CHECK_POINT_DIR = './checkpoint' #saver를 저장할 장소
    
class_names = load_class_names()  #class name을 불러옴
print(class_names)  #['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

vgg= VGG19(None,None)
logits, _=vgg.build_model(X, False)   #vgg model build

cost = vgg.inference_loss(out=logits, t=Y_label) #cost function
train =trainOptimizer(cost = cost) #adamOptimizer를 불러옴

prediction = tf.equal(tf.argmax(logits, 1) , tf.argmax(Y_label, 1))  #prediction
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))  #정확도 계산
    
with tf.Session() as sess:
    last_epoch = tf.Variable(0, name='last_epoch')   #마지막에 돌았던 epoch 저장
    
    sess.run(tf.global_variables_initializer())
    
    saver = restoreSaver(session = sess,       #이전에 돌았던 학습이 있으면 불러옴
                               check_dir = CHECK_POINT_DIR)
    start_from = sess.run(last_epoch)
    
    print("Learinig Start....")
    images_train, cls_train, labels_train = load_training_data()   #training_data load
    images_test, cls_test, labels_test = load_test_data()          #test_data load
    
    batch_size =100 
    batch_start_idx =0          #batch 시작 인덱스
    batch_end_idx = batch_size  #batch 끝 인덱스
    maxidx = images_train.shape[0]
    
    total_batch = int(maxidx/batch_size)  #한 epoch을 돌기 위해 돌아야 하는 횟수
    epoch_run=1000
    print("트레이닝 데이터 수 : ",maxidx)  

    for epoch in range(start_from, epoch_run):       #학습시작
        for step in range(total_batch):
            trainingData = images_train[batch_start_idx:batch_end_idx]     #trainImage[start:end]
            Y = labels_train[batch_start_idx:batch_end_idx]                #label[start:end]
            sess.run(train,feed_dict={X:trainingData, 
                                      Y_label:Y})
            batch_start_idx = (batch_start_idx + batch_size) % maxidx     #한 step 돌면 index에 batch_size를 더해줌
            batch_end_idx = batch_start_idx + batch_size
            
        import random
        random_batch = random.randint(0, len(images_test)-1000)       #test data를 무작위로 받아와서 test함
        acc = sess.run(accuracy,feed_dict={X:images_test[random_batch:random_batch+1000],
                                               Y_label:labels_test[random_batch:random_batch+1000]})
           
        print('===EPOCH %4s===' % (epoch))
        print("[Test]Accuracy at epoch %s: %s" % (epoch, acc))
            
        acc = sess.run(accuracy,feed_dict={X: images_train[batch_start_idx:batch_end_idx], #training acuuracy
                                               Y_label:labels_train[batch_start_idx:batch_end_idx]})
        
        print("[Training]Accuracy at epoch %s: %s" % (epoch, acc))

        saver = saveSaver(session=sess,          #한 epoch 돌 때마다 saver에 여태까지 학습했던 결과를 저장
                          last_epoch=last_epoch,
                          global_step=epoch,
                          saver=saver,
                          check_dir=CHECK_POINT_DIR,
                          model_name='test_model')

Loading data: cifar-10-batches-py/batches.meta
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
conv1a : Tensor("conv1a/Maximum:0", shape=(?, 32, 32, 64), dtype=float32)
conv1b : Tensor("conv1b/Maximum:0", shape=(?, 32, 32, 64), dtype=float32)
max_pool_first : Tensor("MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
conv2a : Tensor("conv2a/Maximum:0", shape=(?, 16, 16, 128), dtype=float32)
conv2b : Tensor("conv2b/Maximum:0", shape=(?, 16, 16, 128), dtype=float32)
max_pool_second : Tensor("MaxPool_1:0", shape=(?, 8, 8, 128), dtype=float32)
conv3a : Tensor("conv3a/Maximum:0", shape=(?, 8, 8, 256), dtype=float32)
conv3b : Tensor("conv3b/Maximum:0", shape=(?, 8, 8, 256), dtype=float32)
conv3c : Tensor("conv3c/Maximum:0", shape=(?, 8, 8, 256), dtype=float32)
conv3d : Tensor("conv3d/Maximum:0", shape=(?, 8, 8, 256), dtype=float32)
max_pool_third : Tensor("MaxPool_2:0", shape=(?, 4, 4, 256), dtype=float32)
conv4a : Tensor("conv4a/Maximum:0", shape=(?